In [113]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [114]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [115]:
tf.keras.backend.clear_session()

In [116]:
subject = 'Alagoas - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [117]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - Consumo de Cimento (t)
0,2003-1,0.635160,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.294434,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,2.359888e+07,30.109
1,2003-2,0.635571,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.296211,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,2.360861e+07,27.684
2,2003-3,0.635982,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.297521,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,2.361833e+07,26.534
3,2003-4,0.636393,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.298766,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,2.362806e+07,25.109
4,2003-5,0.636804,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.299945,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,2.363779e+07,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587645,NaN,24.556360,NaN,NaN,NaN,NaN,46.546
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587084,NaN,24.501314,NaN,NaN,NaN,NaN,47.781
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.585891,NaN,24.609100,NaN,NaN,NaN,NaN,51.171
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.584366,NaN,24.538488,NaN,NaN,NaN,NaN,49.126


In [118]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.106195,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-2.072296,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-2.038397,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-2.004498,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-1.970599,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.358381,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.990315,1.043483,-1.575976,1.179374,-1.254203,1.153065,1.054229
188,1.351367,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.000562,1.037328,-1.527002,1.172432,-1.239868,1.147901,1.048854
189,1.344353,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.011398,1.031173,-1.472984,1.165489,-1.225534,1.142737,1.043480
190,1.337339,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.028853,1.025017,-1.478716,1.158547,-1.211200,1.137574,1.038106


In [119]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 240, dtype: float64

In [120]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.106195,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-2.072296,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-2.038397,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-2.004498,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-1.970599,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.429306,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.348585,1.320458,-0.588269,1.217771,-1.179058,1.109023,1.141942
158,1.428089,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.330705,1.307307,-0.687207,1.221600,-1.195069,1.116989,1.144540
159,1.426872,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.311609,1.294156,-0.785508,1.225429,-1.211080,1.124956,1.147138
160,1.425655,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.291203,1.281004,-0.883126,1.229259,-1.227092,1.132922,1.149736


In [121]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
157    32.321
158    41.937
159    31.982
160    28.499
161    22.626
Name: Alagoas - Consumo de Cimento (t), Length: 162, dtype: float64

In [122]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [123]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [124]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size : split_index + 1]
test_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
126,0.408114,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.871197,1.001453,1.498208,0.788737,-0.193817,0.746412,0.852250
127,0.417817,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.912736,1.019577,1.468126,0.809677,-0.237777,0.767524,0.871358
128,0.427521,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.960653,1.037701,1.431617,0.830616,-0.281737,0.788636,0.890466
129,0.437224,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.006010,1.055825,1.391868,0.851555,-0.325697,0.809748,0.909574
130,0.446928,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.054871,1.073949,1.349013,0.872494,-0.369657,0.830860,0.928682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.358381,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.990315,1.043483,-1.575976,1.179374,-1.254203,1.153065,1.054229
188,1.351367,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.000562,1.037328,-1.527002,1.172432,-1.239868,1.147901,1.048854
189,1.344353,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.011398,1.031173,-1.472984,1.165489,-1.225534,1.142737,1.043480
190,1.337339,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.028853,1.025017,-1.478716,1.158547,-1.211200,1.137574,1.038106


In [125]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [126]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [130]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(36, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [131]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [132]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3080295434, 3411589997, 3493350204, 4126056089, 3952275043, 1329341974, 2752089425, 160634612, 983897875, 3420209789, 3954947174, 3826288614, 3964146437, 2496454483, 3441626604, 351425496, 667673527, 3382532383, 2089902433, 2882535834, 1849207630, 1645913569, 4000609575, 1267592294, 1397538828, 3039703472, 1538026312, 1088171152, 3397338859, 1539246722, 115276956, 323948875, 3487100390, 2514318060, 531693944, 3639962374, 3157115923, 2037036517, 3362258947, 2611757109, 4213376162, 2659144096, 1231563613, 334444739, 1667996347, 2459958409, 577315252, 949972919, 1238355068, 3458637394]


Step: 0 ___________________________________________
val_loss: 24.6981201171875
winner_seed: 3080295434


Step: 1 ___________________________________________
val_loss: 24.783029556274414


Step: 2 ___________________________________________
val_loss: 30.947174072265625


Step: 3 ___________________________________________
val_loss: 29.473703384399414


Step: 4 ___________________________________________
v

In [133]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 196ms/step - loss: 2030.8524 - val_loss: 664.4007
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 775.6238 - val_loss: 342.0463
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 306.0536 - val_loss: 209.7282
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 112.7531 - val_loss: 211.4103
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 115.3351 - val_loss: 169.3457
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 87.5326 - val_loss: 166.3160
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 90.8955 - val_loss: 123.8501
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 87.4892 - val_loss: 139.3809
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 64.3641 - val_loss: 173.9573
Epoch 10/10000
4/4 [==============================] - 0s 9ms/step -

4/4 [==============================] - 0s 10ms/step - loss: 23.3851 - val_loss: 49.3258
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.5198 - val_loss: 51.5276
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.1406 - val_loss: 53.0520
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 20.6168 - val_loss: 66.9040
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 21.6919 - val_loss: 51.7516
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 21.7046 - val_loss: 50.2174
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 23.2691 - val_loss: 73.4738
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.7765 - val_loss: 57.9540
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 20.1068 - val_loss: 50.1029
Epoch 89/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.0784 - val_los

4/4 [==============================] - 0s 10ms/step - loss: 18.5388 - val_loss: 57.3605
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.9097 - val_loss: 56.3590
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.1625 - val_loss: 52.3968
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.1471 - val_loss: 54.3769
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.9940 - val_loss: 53.3682
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.9208 - val_loss: 50.0902
Epoch 165/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.6692 - val_loss: 54.3763
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.2222 - val_loss: 55.8548
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.2522 - val_loss: 53.6638
Epoch 168/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.546

4/4 [==============================] - 0s 10ms/step - loss: 13.7726 - val_loss: 49.5421
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.6275 - val_loss: 52.0495
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.0670 - val_loss: 51.9136
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.5971 - val_loss: 56.3981
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.8805 - val_loss: 58.1815
Epoch 243/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.7949 - val_loss: 63.3701
Epoch 244/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.6050 - val_loss: 53.8128
Epoch 245/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.1549 - val_loss: 55.6311
Epoch 246/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.1892 - val_loss: 55.5307
Epoch 247/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.4049 

4/4 [==============================] - 0s 10ms/step - loss: 16.0558 - val_loss: 49.9714
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.8049 - val_loss: 53.1614
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.1344 - val_loss: 52.4446
Epoch 320/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.2419 - val_loss: 50.3913
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.7430 - val_loss: 49.6009
Epoch 322/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.7784 - val_loss: 50.3107
Epoch 323/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.3493 - val_loss: 57.8980
Epoch 324/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.9328 - val_loss: 49.3046
Epoch 325/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.6049 - val_loss: 49.0043
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.6276 

...

4/4 [==============================] - 0s 9ms/step - loss: 4.7091 - val_loss: 31.4852
Epoch 1902/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.8980 - val_loss: 31.0047
Epoch 1903/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5344 - val_loss: 33.4412
Epoch 1904/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.7458 - val_loss: 34.7737
Epoch 1905/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.9730 - val_loss: 31.6701
Epoch 1906/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0923 - val_loss: 33.6363
Epoch 1907/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9983 - val_loss: 33.4251
Epoch 1908/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3382 - val_loss: 32.0232
Epoch 1909/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.5047 - val_loss: 35.3457
Epoch 1910/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5513 -

4/4 [==============================] - 0s 10ms/step - loss: 5.2880 - val_loss: 33.0601
Epoch 1981/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6229 - val_loss: 32.3512
Epoch 1982/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6076 - val_loss: 36.8511
Epoch 1983/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2546 - val_loss: 32.5728
Epoch 1984/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.5049 - val_loss: 34.4129
Epoch 1985/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.0909 - val_loss: 33.9407
Epoch 1986/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3502 - val_loss: 36.7434
Epoch 1987/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.9070 - val_loss: 38.8505
Epoch 1988/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0225 - val_loss: 34.6652
Epoch 1989/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.5768 -

4/4 [==============================] - 0s 10ms/step - loss: 3.8421 - val_loss: 34.5534
Epoch 2060/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4695 - val_loss: 35.8072
Epoch 2061/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.0005 - val_loss: 35.5638
Epoch 2062/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8319 - val_loss: 41.1057
Epoch 2063/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.4684 - val_loss: 34.7035
Epoch 2064/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.2023 - val_loss: 36.7221
Epoch 2065/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.9678 - val_loss: 34.7060
Epoch 2066/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.2256 - val_loss: 36.7487
Epoch 2067/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7368 - val_loss: 34.9878
Epoch 2068/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0371 -

4/4 [==============================] - 0s 10ms/step - loss: 4.4540 - val_loss: 38.8037
Epoch 2139/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5851 - val_loss: 36.2636
Epoch 2140/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3030 - val_loss: 38.4548
Epoch 2141/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.4142 - val_loss: 36.0206
Epoch 2142/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.9366 - val_loss: 37.5432
Epoch 2143/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.9087 - val_loss: 37.3832
Epoch 2144/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.1996 - val_loss: 35.4932
Epoch 2145/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.9141 - val_loss: 36.3074
Epoch 2146/10000
4/4 [==============================] - 0s 9ms/step - loss: 3.8652 - val_loss: 35.0904
Epoch 2147/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.3900 -

4/4 [==============================] - 0s 9ms/step - loss: 5.2678 - val_loss: 33.1551
Epoch 2218/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6755 - val_loss: 32.2908
Epoch 2219/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.0240 - val_loss: 32.8570
Epoch 2220/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4803 - val_loss: 34.3502
Epoch 2221/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1714 - val_loss: 36.0934
Epoch 2222/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7560 - val_loss: 34.6258
Epoch 2223/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.2212 - val_loss: 33.3448
Epoch 2224/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.5065 - val_loss: 33.8658
Epoch 2225/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.1656 - val_loss: 33.8708
Epoch 2226/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0935 

In [134]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [135]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,29.249224,29.246122,29.180605,29.123461,29.081667,29.048716,29.069643,29.121214,29.134706,29.107325,29.111639,29.155897,29.053225,29.142447,29.17617,29.147556,29.09342,29.021009,29.165657,29.218966,29.25531,29.306639,29.298285,29.326008,29.310362,29.35268,29.331528,29.384802,29.402082,29.424997
Target,21.34,26.848,28.074,29.941,28.977,31.58,29.991,26.82,29.446,27.666,22.013,28.025,25.648,31.782,27.768,33.073,30.389,27.657,32.388,30.378,30.641,32.337,36.405,22.499,27.961,32.378,32.582,42.703,38.584,36.503
Error,7.909224,2.398123,1.106606,0.817539,0.104668,2.531284,0.921356,2.301214,0.311293,1.441324,7.098639,1.130898,3.405224,2.639553,1.408171,3.925446,1.29558,1.36401,3.222343,1.159035,1.385691,3.030363,7.106714,6.827007,1.349361,3.025318,3.250473,13.318197,9.181917,7.078001


In [136]:
display(mae)
display(mape)

3.401486

0.11300255

In [137]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [138]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[349.63022]] - Target[330.72099999999995]| =  Error: [[18.90921]]; MAPE:[[0.05717571]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[350.20468]] - Target[360.96500000000003]| =  Error: [[10.760315]]; MAPE:[[0.02980986]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[176.20645]] - Target[210.711]| =  Error: [[34.504547]]; MAPE:[[0.16375294]]


[array([[18.90921]], dtype=float32),
 array([[10.760315]], dtype=float32),
 array([[34.504547]], dtype=float32)]

21.391357

0.0835795